* The temporal order is scrambled in the test data making TS useless there.
* Still, just for learning / realism, we can still do it in the training data! 
* Let's add pseudo dates, and aggregate features on column subsets. Finally i'll run a model to predict the target!

In [1]:
import importlib
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
# Temporary for not Kaggle data partitioning
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from datetime import datetime
from scipy.special import logsumexp

from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import mean_squared_error, classification_report

In [2]:
train = pd.read_csv("caltech-cs155-2020/train.csv")
test = pd.read_csv("caltech-cs155-2020/test.csv")
df = pd.concat([train,test],sort=False)
print(df.shape)
print(df.columns)
df.tail()

(784239, 28)
Index(['id', 'last_price', 'mid', 'opened_position_qty ',
       'closed_position_qty', 'transacted_qty', 'd_open_interest', 'bid1',
       'bid2', 'bid3', 'bid4', 'bid5', 'ask1', 'ask2', 'ask3', 'ask4', 'ask5',
       'bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol', 'ask1vol',
       'ask2vol', 'ask3vol', 'ask4vol', 'ask5vol', 'y'],
      dtype='object')


,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol,y
191854,784234,5150.6,5150.8,NaN,NaN,0.0,0,5150.6,5150.4,5150.2,...,5,3,4,2,1,3,1,3,2,NaN
191855,784235,5122.6,5122.7,1.0,2.0,3.0,-1,5122.6,5122.4,5122.2,...,1,5,1,20,16,17,4,1,16,NaN
191856,784236,5192.8,5192.3,NaN,NaN,0.0,0,5192.0,5191.8,5191.2,...,2,1,1,1,1,2,1,2,1,NaN
191857,784237,5152.0,5151.9,4.0,2.0,6.0,2,5151.8,5151.6,5151.4,...,5,6,8,37,7,1,2,1,1,NaN
191858,784238,5200.0,5200.3,2.0,0.0,2.0,2,5200.0,5199.0,5198.8,...,5,1,2,1,1,1,4,1,3,NaN


In [3]:
train.head()

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol,y
0,0,3842.4,3842.6,NaN,NaN,103.0,0,3842.4,3842.0,3841.8,...,1,6,14,6,6,1,1,10,2,1
1,1,3842.8,3843.4,6.0,49.0,55.0,-43,3843.0,3842.8,3842.4,...,6,11,1,6,1,4,4,1,13,0
2,2,3844.0,3844.3,7.0,77.0,84.0,-69,3843.8,3843.6,3843.2,...,1,4,21,12,1,16,10,4,9,0
3,3,3843.8,3843.4,3.0,34.0,37.0,-30,3843.0,3842.8,3842.4,...,13,12,2,4,2,7,1,2,11,1
4,4,3843.2,3843.1,3.0,38.0,41.0,-35,3842.8,3842.4,3842.0,...,12,2,2,4,1,3,1,11,15,1


In [4]:
test.head()

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,bid1vol,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol
0,592380,5178.4,5178.3,1.0,11.0,12.0,-10,5178.2,5178.0,5177.8,...,3,16,3,1,1,1,4,1,5,2
1,592381,5133.0,5132.9,1.0,0.0,1.0,1,5132.8,5132.6,5132.4,...,1,2,2,1,10,4,8,1,1,1
2,592382,5177.4,5178.0,2.0,0.0,2.0,2,5177.2,5176.8,5176.6,...,8,18,1,1,6,7,4,1,2,5
3,592383,5093.6,5093.9,0.0,2.0,2.0,-1,5093.2,5093.0,5092.8,...,8,5,4,3,1,1,1,11,6,2
4,592384,5189.2,5189.2,NaN,NaN,0.0,0,5188.8,5188.6,5188.4,...,1,5,1,3,3,3,2,1,3,4


In [5]:
## y is binary.
display(train["y"].describe())

count    592380.000000
mean          0.356958
std           0.479103
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
Name: y, dtype: float64

In [6]:
bid_cols = ['bid1','bid2', 'bid3', 'bid4', 'bid5']
bid_vol_cols = ['bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol']
ask_cols = ['ask1', 'ask2', 'ask3', 'ask4', 'ask5',]
ask_vol_cols = ['ask1vol','ask2vol', 'ask3vol', 'ask4vol', 'ask5vol']

group_cols = {"bid_cols":bid_cols,"bid_vol_cols":bid_vol_cols,"ask_cols":ask_cols,"ask_vol_cols":ask_vol_cols}

* Additional features could include: rank, which bid number is the max/min, etc' 
* features between the aggregated features (e.g. max bid div max ask..)

In [7]:
for group in group_cols.keys():
    print(group)
    df[f"{group}_max"] = df[group_cols[group]].max(axis=1)
    df[f"{group}_min"] = df[group_cols[group]].min(axis=1)
    df[f"{group}_spread"] = df[f"{group}_max"].div(df[f"{group}_min"])
#     df[f"{group}_logsumexp"] = df[grou/_cols[group]].apply(logsumexp)
    
    df[f"{group}_max"] = df[group_cols[group]].max(axis=1)
    
df["last_price_div__mid"] = df["last_price"].div(df["mid"])

bid_cols
bid_vol_cols
ask_cols
ask_vol_cols


In [8]:
df["date"] = pd.to_datetime("1.1.2019")
df["date"] = df["date"] + pd.to_timedelta(df["id"]/2,unit="s") # 500 ms per row

df["date"].describe()

count                         784239
unique                        784239
top       2019-01-02 14:21:54.500000
freq                               1
first            2019-01-01 00:00:00
last             2019-01-05 12:55:19
Name: date, dtype: object

# Split back into train and test, and build model

In [9]:
train = df.loc[~df.y.isna()]
print(f"train shape {train.shape[0]}")
test = df.loc[df.y.isna()]
print(f"test shape {test.shape[0]}")

train shape 592380
test shape 191859


In [10]:
train_no_nan = train.fillna(train.mean())
test_no_nan = test.fillna(test.mean())

In [11]:
train_no_nan.isna()

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,bid_vol_cols_min,bid_vol_cols_spread,ask_cols_max,ask_cols_min,ask_cols_spread,ask_vol_cols_max,ask_vol_cols_min,ask_vol_cols_spread,last_price_div__mid,date
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592375,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
592376,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
592377,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
592378,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [12]:
train.drop(["id"],axis=1).to_csv("train_hft.csv.gz",index=False,compression="gzip")
test.to_csv("test_hft_nodates.csv.gz",index=False,compression="gzip")

In [13]:
# we don't know if the test set has a temporal split, so we'll just try a random split for now
X = train_no_nan.drop(["id","date","y"],axis=1)
y = train_no_nan["y"]

In [14]:
print(X.shape)
print(np.isfinite(X).all())

(592380, 39)
last_price              True
mid                     True
opened_position_qty     True
closed_position_qty     True
transacted_qty          True
d_open_interest         True
bid1                    True
bid2                    True
bid3                    True
bid4                    True
bid5                    True
ask1                    True
ask2                    True
ask3                    True
ask4                    True
ask5                    True
bid1vol                 True
bid2vol                 True
bid3vol                 True
bid4vol                 True
bid5vol                 True
ask1vol                 True
ask2vol                 True
ask3vol                 True
ask4vol                 True
ask5vol                 True
bid_cols_max            True
bid_cols_min            True
bid_cols_spread         True
bid_vol_cols_max        True
bid_vol_cols_min        True
bid_vol_cols_spread     True
ask_cols_max            True
ask_cols_min            True
a

# For Random Forest Model

In [ ]:
#model = CatBoostClassifier()
from sklearn.ensemble import RandomForestClassifier
import randomforestorder
importlib.reload(randomforestorder)
model = randomforestorder.RandomForestOrder()
model.fit(X,y)

# For Neural Network Model 1

In [15]:
#model = CatBoostClassifier()
import torch
import torch.nn as nn
import first_neural_network
importlib.reload(first_neural_network)
model = first_neural_network.first_neural_network()
# model.fit(X,y)

In [ ]:
# print(X_temp.shape)

In [ ]:
# from scipy import stats
# X_temp = np.asarray(X)
# print(stats.zscore(X_temp, axis=0))


In [ ]:
# print(type(X))
# print(type(y))
# X_temp = np.asarray(X)
# print((X_temp.shape))
# print(np.std(X_temp, axis=0).shape)
# print(np.mean(X_temp, axis=0).shape)
# std_dev = np.std(X_temp, axis=0)
# mean_val = np.mean(X_temp, axis=0)
# z_score = X_temp

# For Neural Network Model 2

In [ ]:
#model = CatBoostClassifier()
import second_neural_network
importlib.reload(second_neural_network)
model = second_neural_network.second_neural_network()
model.fit(X,y)

In [16]:
from sklearn.metrics import log_loss

y_pred = model.predict(train_no_nan.drop(["id","date","y"],axis=1))
print(log_loss(y, y_pred))


0.7393539995942637


In [17]:
print(type(y))
print(type(y_pred))
print(y.shape)
print(y_pred.shape)

<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
(592380,)
(592380, 1)


In [18]:
print(y_pred[0:100])
print(y[0:100])

[[0.62841475]
 [0.6561799 ]
 [0.6978448 ]
 [0.6413466 ]
 [0.66529185]
 [0.593712  ]
 [0.5888055 ]
 [0.660815  ]
 [0.616337  ]
 [0.5891048 ]
 [0.5957049 ]
 [0.5889407 ]
 [0.59142774]
 [0.6107769 ]
 [0.55831885]
 [0.59162253]
 [0.56636393]
 [0.57468975]
 [0.5838935 ]
 [0.5712798 ]
 [0.6026359 ]
 [0.57722706]
 [0.58416563]
 [0.5798602 ]
 [0.6054348 ]
 [0.59901583]
 [0.5951816 ]
 [0.56604993]
 [0.58722955]
 [0.59015703]
 [0.6099125 ]
 [0.6050743 ]
 [0.573032  ]
 [0.58724236]
 [0.59339803]
 [0.5928702 ]
 [0.56719667]
 [0.5630235 ]
 [0.57213265]
 [0.5853204 ]
 [0.5565242 ]
 [0.57753867]
 [0.59437084]
 [0.56408954]
 [0.5887029 ]
 [0.5775476 ]
 [0.58278596]
 [0.59098333]
 [0.58644   ]
 [0.57488877]
 [0.59276193]
 [0.57099885]
 [0.5830294 ]
 [0.6240595 ]
 [0.5914348 ]
 [0.5807218 ]
 [0.5806488 ]
 [0.57195777]
 [0.582576  ]
 [0.5893809 ]
 [0.5663586 ]
 [0.59469837]
 [0.56457424]
 [0.5736196 ]
 [0.5599493 ]
 [0.5870368 ]
 [0.5860037 ]
 [0.57006115]
 [0.5826752 ]
 [0.5835474 ]
 [0.5778983 ]
 [0.57

In [ ]:
# print(np.sum(np.abs((y - y_pred))/y.shape[0])

In [ ]:
# y_prob = model.predict(train_no_nan.drop(["id","date","y"],axis=1))

In [ ]:
import get_average_val_err

importlib.reload(get_average_val_err)

print(get_average_val_err.get_val_err(3, train_no_nan, model))

592380
Fold  1  of  3  test indices: [     0      1      2 ... 197457 197458 197459]
len(val_index_list):  197460
x train tensor:  tensor([[-2.1965e+00, -2.1984e+00, -7.3214e-01,  ..., -5.4239e-01,
         -2.0939e-01,  4.7410e-01],
        [-2.1965e+00, -2.1965e+00, -2.1626e-01,  ...,  5.0589e-01,
         -6.8383e-01, -3.0243e-03],
        [-2.1928e+00, -2.1947e+00, -2.1626e-01,  ...,  5.0589e-01,
         -6.8383e-01,  4.7407e-01],
        ...,
        [ 1.7876e+00,  1.8080e+00, -9.6102e-03,  ...,  1.5542e+00,
         -7.5161e-01, -4.9756e+00],
        [ 1.7876e+00,  1.8080e+00, -9.6102e-03,  ...,  1.5542e+00,
         -7.5161e-01, -4.9756e+00],
        [ 1.7876e+00,  1.8080e+00, -9.6102e-03,  ...,  1.5542e+00,
         -7.5161e-01, -4.9756e+00]])
y train tensor:  tensor([[1.],
        [1.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])
Train Epoch: 1  Loss: 0.9793
Train Epoch: 2  Loss: 0.4535
Train Epoch: 3  Loss: 0.6527
Train Epoch: 4  Loss: 0.5473
Train 

In [ ]:
# model.fit(X,y)

## export predictions

In [ ]:
# test["Predicted"] = model.predict(test.drop(["id","date","y"],axis=1),prediction_type='Probability')[:,1]
test["Predicted"] = model.predict(test_no_nan.drop(["id","date","y"],axis=1))
test[["id","Predicted"]].to_csv("sabera_submission_25_epochs_test.csv",index=False)